In [3]:
%%writefile graph_traversal.cu
#include <iostream>
#include <queue>
#include <stack>

using namespace std;

#define V 6  // Number of vertices


__device__ int adj_matrix[V][V] = {
    {0, 0, 1, 1, 0},
    {0, 0, 1, 0, 1},
    {1, 1, 0, 0, 0},
    {1, 1, 0, 0, 1},
    {0, 1, 0, 1, 0},
};

__global__ void cudaBFS(int* visited, int start) {
    if (threadIdx.x == 0 && blockIdx.x == 0) {
        visited[start] = 1;
        printf("%d ", start); // Print the start vertex

        int queue[V];
        int front = 0, rear = 0;
        queue[rear++] = start;

        while (front < rear) {
            int current = queue[front++];

            for (int j = 0; j < V; ++j) {
                if (adj_matrix[current][j] && !visited[j]) {
                    visited[j] = 1;
                    printf("%d ", j); // Print the visited vertex
                    queue[rear++] = j;
                }
            }
        }
    }
}


__device__ void recursiveDFS(int* visited, int vertex) {
    visited[vertex] = 1;
    printf("%d ", vertex); // Print the current vertex

    for (int j = 0; j < V; ++j) {
        if (adj_matrix[vertex][j] && !visited[j]) {
            recursiveDFS(visited, j);
        }
    }
}

__global__ void cudaDFS(int* visited, int start) {
    recursiveDFS(visited, start);
}



int main() {
    int start_vertex = 0;  // Starting vertex for traversal

    // Allocate memory for visited array on the host and initialize with zeros
    int* visited = new int[V];
    for (int i = 0; i < V; ++i) {
        visited[i] = 0;
    }

    // Allocate memory for visited array on the device
    int* d_visited;
    cudaMalloc((void**)&d_visited, V * sizeof(int));
    cudaMemcpy(d_visited, visited, V * sizeof(int), cudaMemcpyHostToDevice);

    // Breadth-First Search (BFS)
    cout << "BFS traversal: ";
    cudaBFS<<<1, V>>>(d_visited, start_vertex);
    cudaMemcpy(visited, d_visited, V * sizeof(int), cudaMemcpyDeviceToHost);
    cout << endl;

    // Reset visited array
    for (int i = 0; i < V; ++i) {
        visited[i] = 0;
    }
    cudaMemcpy(d_visited, visited, V * sizeof(int), cudaMemcpyHostToDevice);

    // Depth-First Search (DFS)
    cout << "DFS traversal: ";
    cudaDFS<<<1, 1>>>(d_visited, start_vertex);
    cudaMemcpy(visited, d_visited, V * sizeof(int), cudaMemcpyDeviceToHost);
    cout << endl;

    // Free device memory
    cudaFree(d_visited);

    // Clean up host memory
    delete[] visited;

    return 0;
}


Writing graph_traversal.cu


In [4]:
!nvcc graph_traversal.cu -o bfs

ptxas warning : Stack size for entry function '_Z7cudaDFSPii' cannot be statically determined


In [5]:
!./bfs

BFS traversal: 0 2 3 1 4 
DFS traversal: 0 2 1 4 3 
